
# Быстрый старт с Policyscope

Этот ноутбук показывает на простом примере, как оценить новую рекомендательную политику без запуска дорогого онлайн‑эксперимента. Мы сгенерируем синтетических пользователей, сравним две политики и применим методы Replay, IPS, SNIPS и Doubly Robust.



## Алгоритмы

* **Replay** — усредняет отклики только по тем случаям, когда новая политика сделала бы то же действие, что и логирующая. Дисперсия может быть высокой.
* **IPS** — взвешивает каждый лог по отношению вероятностей в новой и старой политике. Даёт несмещённую оценку, но чувствителен к большим весам.
* **SNIPS** — нормализует веса IPS и часто даёт более стабильную оценку.
* **Doubly Robust** — объединяет модель отклика с IPS. Остаётся несмещённым, если верны либо пропенсити, либо модель отклика.



## 1. Генерация данных

Создадим пользователей и посмотрим на их признаки. Суффикс `_z` означает нормализованное значение (z‑score).


In [1]:

from policyscope.synthetic import SynthConfig, SyntheticRecommenderEnv

cfg = SynthConfig(n_users=5000, horizon_days=30, seed=0)
env = SyntheticRecommenderEnv(cfg)
X = env.sample_users()
X.head()


,user_id,loyal,age,risk,income,region,age_z,risk_z,income_z
0,0,0,53,0.528127,23405.298109,0,1.083333,0.112508,-0.888211
1,1,1,64,0.412230,37336.939988,4,2.000000,-0.351079,0.050984
2,2,1,43,0.217916,59992.063224,0,0.250000,-1.128335,1.004675
3,3,1,53,0.948469,32836.001473,4,1.083333,1.793876,-0.207349
4,4,0,56,0.576872,41900.225286,0,1.333333,0.307490,0.282872



## 2. Определяем политики

Политика **A** генерирует логи и делает случайный шаг в 10% случаев (epsilon‑greedy). Политика **B** отдаёт предпочтение более прибыльным действиям по правилу softmax.


In [2]:

from policyscope.policies import make_policy

policyA = make_policy("epsilon_greedy", epsilon=0.1)
policyB = make_policy("softmax", tau=0.7)



## 3. Логи политики A

Симулируем поведение политики A и посмотрим на несколько первых строк.


In [3]:

logsA = env.simulate_logs_A(policyA, X)
logsA.head()


,user_id,loyal,age,risk,income,region,age_z,risk_z,income_z,a_A,propensity_A,accept,cltv
0,0,0,53,0.528127,23405.298109,0,1.083333,0.112508,-0.888211,1,0.925,0,168.199352
1,1,1,64,0.412230,37336.939988,4,2.000000,-0.351079,0.050984,0,0.925,1,1225.575479
2,2,1,43,0.217916,59992.063224,0,0.250000,-1.128335,1.004675,3,0.925,1,948.090268
3,3,1,53,0.948469,32836.001473,4,1.083333,1.793876,-0.207349,0,0.925,1,960.910888
4,4,0,56,0.576872,41900.225286,0,1.333333,0.307490,0.282872,2,0.925,1,510.971281



Столбец `a_A` — показанное действие, `propensity_A` — вероятность его показа. `accept` и `cltv` — отклик и ценность пользователя.



## 4. Истинные значения метрики

Посчитаем истинные вероятности отклика для обеих политик, чтобы сравнить их с оценками.


In [4]:

V_true_A = env.oracle_value(policyA, X, metric="accept")
V_true_B = env.oracle_value(policyB, X, metric="accept")
print(f"Истинное V(A): {V_true_A:.3f}")
print(f"Истинное V(B): {V_true_B:.3f}")


Истинное V(A): 0.463
Истинное V(B): 0.477



## 5. Оценка новой политики по логам A

Сначала вычислим значение политики A на собственных логах, затем сравним с оценками политики B разными методами.


In [5]:

from policyscope.estimators import (
    value_on_policy,
    prepare_piB_taken,
    ips_value,
    snips_value,
    dr_value,
    train_mu_hat,
)

piB_taken = prepare_piB_taken(logsA, policyB)
V_A = value_on_policy(logsA, target="accept")
V_B_ips, _, _ = ips_value(logsA, piB_taken, target="accept")
V_B_snips, _, _ = snips_value(logsA, piB_taken, target="accept")
mu_hat = train_mu_hat(logsA, target="accept")
V_B_dr, _, _ = dr_value(logsA, policyB, mu_hat, target="accept")

print(f"On-policy V(A): {V_A:.3f}")
print(f"IPS V(B): {V_B_ips:.3f}")
print(f"SNIPS V(B): {V_B_snips:.3f}")
print(f"DR V(B): {V_B_dr:.3f}")


On-policy V(A): 0.477
IPS V(B): 0.469
SNIPS V(B): 0.474
DR V(B): 0.475



## Итоги

В этом примере оценки IPS, SNIPS и DR близки к истинному значению \(V(B)\). На практике это позволяет приблизительно понять эффект новой политики, не проводя A/B‑тест.



## Ссылки

- [Counterfactual Evaluation for Recommendation Systems](https://eugeneyan.com/writing/offline-recsys/)
- Farajtabar et al., *More Robust Doubly Robust Off-policy Evaluation* (arXiv:2205.13421)
